This YOLOv8 🚀 notebook by Vitis AI presents simple train, validate and predict examples to help start your AI adventure.
We hope that the resources in this notebook will help you get the most out of YOLOv8.

# Setup and Prepare

Clone GitHub repository, install dependencies and check PyTorch and GPU.

## Prepare the environment

In [2]:
%pip install -r requirements.txt

Looking in indexes: https://xcdpython.xilinx.com/simple
...



In [4]:
%cd code/
!python setup.py develop

running egg_info
writing ultralytics.egg-info/PKG-INFO
writing dependency_links to ultralytics.egg-info/dependency_links.txt
writing entry points to ultralytics.egg-info/entry_points.txt
writing requirements to ultralytics.egg-info/requires.txt
writing top-level names to ultralytics.egg-info/top_level.txt
reading manifest file 'ultralytics.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'ultralytics.egg-info/SOURCES.txt'
running build_ext
Removing ultralytics 8.0.32 from easy-install.pth file
Adding ultralytics 8.0.32 to easy-install.pth file
Processing dependencies for ultralytics==8.0.32
Searching for setuptools>=18.5
...


## Prepare the dataset

##### Download COCO2017 dataset.(refer to this repo https://github.com/ultralytics/ultralytics)

### Put coco2017 dataset under the ./code/data directory, dataset directory structure like:
```markdown
+ data/
    + coco/
        + labels/
        + annotations/
        + images/
        + test-dev2017.txt 
        + train2017.txt
        + val2017.txt
```
### Modify the path of coco in coco.yaml to your custom dataset
```
path: /path/to/the/coco_dataset  # dataset root dir
train: train2017.txt
val: val2017.txt
```

# Train/Validation

## Train

In [12]:
# for float training
%cd code
!CUDA_VISIBLE_DEVICES=0 yolo detect train data="datasets/coco.yaml" model=./../float/yolov8m.pt pretrained=True epochs=50 batch=2 device=0

[VAIQ_NOTE]: Loading NNDCT kernels...

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.Conv                  [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.Conv                  [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.C2f                   [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.Conv                  [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.C2f                   [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.Conv                  [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.C2f                   [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

## Test

In [13]:
%cd code
!CUDA_VISIBLE_DEVICES=0 yolo detect val data="datasets/coco.yaml" model=./../float/yolov8m.pt

[VAIQ_NOTE]: Loading NNDCT kernels...

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.Conv                  [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.Conv                  [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.C2f                   [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.Conv                  [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.C2f                   [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.Conv                  [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.C2f                   [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

## Quant

In [15]:
!CUDA_VISIBLE_DEVICES=0 yolo detect val data="datasets/coco.yaml" model=./../float/yolov8m.pt \
    nndct_quant=True quant_mode=calib --nndct_convert_sigmoid_to_hsigmoid --nndct_convert_silu_to_hswish

!CUDA_VISIBLE_DEVICES=0 yolo detect val data="datasets/coco.yaml" model=./../float/yolov8m.pt \
    nndct_quant=True quant_mode=test --nndct_convert_sigmoid_to_hsigmoid --nndct_convert_silu_to_hswish

[VAIQ_NOTE]: Loading NNDCT kernels...

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.Conv                  [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.Conv                  [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.C2f                   [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.Conv                  [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.C2f                   [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.Conv                  [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.C2f                   [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

### Performance
| Model             | Input Size | Float mAP   | Quant mAP   | FLOPs  |
|-------------------|------------|-------------|-------------|--------|
| YOLOv8m           | 640*640    | 50.0%       | 48.7%       | 78.9G  |